In [1]:
epochs, batch, lr = 100, 32, 1e-4
num_clients, in_channels, num_classes = 20, 1, 10
Dataset, Model, Prefix = 'MNIST', 'CNN_3', 'Parallel'
Gdrive = '/content/drive/MyDrive/ColabNotebooks/SplitLearning3'
import torch; torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40, uuid=c9a56b3f-7ee9-5b4e-f01c-90d0ac2b3616, L2_cache_size=4MB)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os; os.chdir(Gdrive)
%run library/Dataset.ipynb
%run library/Network.ipynb
%run library/Training.ipynb
from torchsummary import summary

In [4]:
!pip install pytorch-msssim
from pytorch_msssim import ssim, SSIM

In [5]:
def train_parallel(clients, testset, epochs=epochs, model=Model, batch=batch, lr=lr):
  in_channels, num_classes, tester = loadTester(testset)
  models = [globals()[model](in_channels, num_classes) for c in clients]
  model0 = SplitNN([Client(m.client) for m in models[:int(len(clients)/2)]], Server(models[0].server))
  model1 = SplitNN([Client(m.client) for m in models[int(len(clients)/2):]], Server(models[1].server))
  model0.initialize(clients[:int(len(clients)/2)], tester, epochs, batch, lr)
  model1.initialize(clients[int(len(clients)/2):], tester, epochs, batch, lr)

  for epoch in tqdm(range(epochs)):
    model0.train_network(epoch, private=True, sequence=False, federate=False)
    model1.train_network(epoch, private=True, sequence=False, federate=False)
    server0, server1 = model0.server.state_dict(), model1.server.state_dict()
    for key in server0:
      server0[key] += server1[key]
      if server0[key].type().split('.')[-1] == 'LongTensor':
        torch.div(server0[key], 2, rounding_mode='floor')
      else: server0[key] /= 2
    model1.server.load_state_dict(server0)
    model0.evaluate(), model1.evaluate()
    plot_progress(model0.clients + model1.clients, locals())
  return model0, model1

In [6]:
Dataset = 'Fashion'
dataset, testset = loadDataset(Dataset)

100%|██████████| 26.4M/26.4M [00:02<00:00, 13.0MB/s]


Extracting /content/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]


Extracting /content/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.46MB/s]


Extracting /content/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 19.8MB/s]

Extracting /content/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



In [7]:
Model = 'VGG_7'
clients = torch.ones((num_clients,))
clients = ratio_split(dataset, clients / clients.sum())

In [ ]:
Model = 'VGG_7'
clients = torch.ones((num_clients,))
clients = ratio_split(dataset, clients / clients.sum())
_ = train_splitnn(clients, (testset), 'private', 50, model=Model)

  0%|          | 0/50 [00:00<?, ?it/s]

Output()

In [ ]:
caches = [1 for i in range(len(clients))]
network = train_splitnn(clients, (testset), 'private', 50, model=Model, caches=caches)

  0%|          | 0/50 [00:00<?, ?it/s]

Output()

In [ ]:
class Decoder(Autodecoder):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, 2, 1, 1), nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, 1, 1), nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
def reconstruct2(decoders):
    similarities = [[] for i in range(num_clients)]
    for images, _ in tqdm(DataLoaderGPU(testset, batch * 2)):
        for i in range(num_clients):
            with torch.no_grad():
                network.clients[i].eval(), decoders[i].eval()
                output = decoders[i](network.clients[i](images))
            similarities[i].append(ssim(images, output, 1.0).item())
    for sim in similarities:
        print(np.mean(sim))

In [ ]:
decoder = Decoder().cuda()
decoder.train_model(network.clients[0], network.clients[0], 30)
reconstruct2([decoder for i in range(num_clients)])

  0%|          | 0/30 [00:00<?, ?it/s]

Output()

  0%|          | 0/157 [00:00<?, ?it/s]

0.9577415653854419
0.5415077721996672
0.6084659976564395
0.5488018473242499
0.5493064200042919
0.5463680583200637
0.5733173257985692
0.6463276219975417
0.6004391187315534
0.6082534774853166
0.563228875968107
0.6243227754428888
0.5177395243173951
0.5273464627706321
0.5556707450538684
0.6562936814727297
0.6165341321070483
0.5730754398995903
0.620928685756246
0.6504842479517505


In [ ]:
network = train_splitnn(clients, (testset), 'sequence', 50, model=Model)

  0%|          | 0/50 [00:00<?, ?it/s]

Output()

In [ ]:
network = train_splitnn(clients, (testset), 'federate', 50, model=Model)

  0%|          | 0/50 [00:00<?, ?it/s]

Output()

In [ ]:
network = train_parallel(clients, (testset,1,10), model=Model)

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [ ]:
def train_parallelX(clients, testset, epochs=epochs, model=Model, batch=batch, lr=lr, X=2):
    in_channels, num_classes, tester = loadTester(testset)
    step = len(clients) // X
    models = [globals()[model](in_channels, num_classes) for c in clients]
    modelX = [SplitNN([Client(m.client) for m in models[i*step:(i+1)*step]], Server(models[i].server)) for i in range(X)]
    for i in range(X): modelX[i].initialize(clients[i*step:(i+1)*step], tester, epochs, batch, lr)

    for epoch in tqdm(range(epochs)):
        for model in modelX: model.train_network(epoch, private=True, sequence=False, federate=False)
        server = modelX[0].server.state_dict()
        for key in server:
            for i in range(1, X):
                server[key] += modelX[i].server.state_dict()[key]
            if server[key].type().split('.')[-1] == 'LongTensor':
                torch.div(server[key], X, rounding_mode='floor')
            else: server[key] /= X
        clients = []
        for i in range(X):
            modelX[i].server.load_state_dict(server)
            modelX[i].evaluate()
            clients += modelX[i].clients
        plot_progress(clients, locals())

In [ ]:
network = train_parallelX(clients, (testset,1,10), model=Model, X=4)

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [ ]:
network = train_parallelX(clients, (testset,1,10), 150, model=Model, X=4)

  0%|          | 0/150 [00:00<?, ?it/s]

Output()

In [ ]:
network = train_parallelX(clients, (testset,1,10), 150, model=Model, X=5)

  0%|          | 0/150 [00:00<?, ?it/s]

Output()

In [ ]:
network = train_parallelX(clients, (testset,1,10), 150, model=Model, X=10)

  0%|          | 0/150 [00:00<?, ?it/s]

Output()

In [ ]:
network = train_parallelX(clients, (testset,1,10), 150, model=Model, X=20)

  0%|          | 0/150 [00:00<?, ?it/s]

Output()

In [ ]:
network = train_parallelX(clients, (testset,1,10), 200, model=Model, X=20)

  0%|          | 0/200 [00:00<?, ?it/s]

Output()

In [17]:
import random
def train_parallelXY(clients, testset, epochs=epochs, model=Model, batch=batch, lr=lr, X=2, Y=[2]):
    in_channels, num_classes, tester = loadTester(testset)
    step = len(clients) // X
    models = [globals()[model](in_channels, num_classes) for c in clients]
    modelX = [SplitNN([Client(m.client) for m in models[i*step:(i+1)*step]], Server(models[i].server)) for i in range(X)]
    for i in range(X): modelX[i].initialize(clients[i*step:(i+1)*step], tester, epochs, batch, lr)

    for epoch in tqdm(range(epochs)):
        for model in modelX: model.train_network(epoch, private=True, sequence=False, federate=False)
        server = modelX[0].server.state_dict()
        for key in server:
            for i in range(1, X):
                server[key] += modelX[i].server.state_dict()[key]
            if server[key].type().split('.')[-1] == 'LongTensor':
                torch.div(server[key], X, rounding_mode='floor')
            else: server[key] /= X
        clients = []
        for i in range(X):
            modelX[i].server.load_state_dict(server)
            modelX[i].evaluate()
            clients += modelX[i].clients
        plot_progress(clients, locals())
        if epoch > 0 and epoch % 10 == 0:
            clients = []
            for i in range(X): clients.extend(modelX[i].clients)
            random.shuffle(Y); X = Y[0]
            step = len(clients) // X
            modelX = [SplitNN(clients[i*step:(i+1)*step], Server(models[i].server)) for i in range(X)]
            for i in range(X): modelX[i].cached = [[] for c in modelX[i].clients]
            for i in range(X): modelX[i].server.initialize(tester, epochs, lr)
            for i in range(X): modelX[i].server.load_state_dict(server)
            print('Change to', X, 'servers')
        PingColab()

network = train_parallelXY(clients, (testset,1,10), epochs=150, model=Model, X=2, Y=[2,4,5,10])

  0%|          | 0/150 [00:00<?, ?it/s]

Output()

Change to 2 servers
Change to 10 servers
Change to 5 servers
Change to 4 servers
Change to 2 servers
Change to 5 servers
Change to 2 servers
Change to 4 servers
Change to 5 servers
Change to 2 servers
Change to 2 servers
Change to 10 servers
Change to 4 servers
Change to 4 servers
